In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal, interpolate
import copy
import os 
import pandas as pd
os.chdir('/home/kkotzen/research/PPG_sleepstaging/')
from pathlib import Path
from src.parsing.MESAParser import MESAParser
from src.parsing.utils.comp_ppg_signal_quality import ppg_window_quality, ppg_window_quality_summary, moving_average_filter
from src.parsing.armand.feature_comp import bsqi
from tqdm import tqdm
def duplicate(x):
    return np.array([[x_]*2 for x_ in x]).flatten()

def hold_line(t, x):
    print(t.shape, x.shape)
    t_ = duplicate(t)[1:-1]
    x_ = duplicate(x)
    return t_, x_

def calculate_ptt(ppg_peaks, ecg_peaks, fs=256, max_ptt=0.65, min_ptt=0.15, smoothing_length=1000):
    ptt = np.zeros_like(ecg_peaks)
    ptt[:] = 100

    for i, ecg_peak in enumerate(ecg_peaks):
        try:
            times = ppg_peaks - ecg_peak
            times = times[times > 0][0]
            if (times < max_ptt*fs and times > min_ptt*fs):
                ptt[i] = times
        except:
            if i > 0:
                ptt[i] = ptt[i - 1]

    ptt = signal.filtfilt(np.ones(smoothing_length) / smoothing_length, 1, ptt, padlen=smoothing_length)
    ptt = np.array(ptt[0:len(ecg_peaks)]).astype(int)
    return ptt

def calculate_delayed_ecg(ppg_peaks, ecg_peaks):
    return ecg_peaks + calculate_ptt(ppg_peaks, ecg_peaks)

In [2]:
dl = MESAParser()
patients = ['0001'] #dl.database_all_patient_IDs

ppg = dl.load_signal(patients[0], 'Pleth')
len_sig = len(ppg)
ecg_peaks =  dl.load_annotation(patients[0], 'EKG', 'epltd0', 'Peaks')
ppg_peaks_ref = dl.load_annotation(patients[0], 'Pleth', 'Aboy', 'Peaks').astype(int)
ppg_peaks_comp = dl.load_annotation(patients[0], 'Pleth', 'Bishop', 'Peaks').astype(int)
ecg_peaks = calculate_delayed_ecg(ppg_peaks_ref, ecg_peaks).astype(int)

In [3]:
plt.close("all")
plt.plot(ppg)
plt.plot(ppg_peaks_ref, ppg[ppg_peaks_ref], 'x')
plt.plot(ppg_peaks_comp, ppg[ppg_peaks_comp], 'o')
plt.plot(ecg_peaks, ppg[ecg_peaks], '*')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
window = 30
fs = 256
window_fs = window * fs
epochs = np.arange(0, int(len_sig / window_fs), 1)
feats = pd.DataFrame()
for epoch in tqdm(epochs):
    this_peaks_ref = ppg_peaks_ref[(ppg_peaks_ref >= (epoch)* window_fs) * 
                                   (ppg_peaks_ref < (epoch+1) * window_fs)]
    this_peaks_comp = ppg_peaks_comp[(ppg_peaks_comp >= (epoch)* window_fs) * 
                                     (ppg_peaks_comp < (epoch+1) * window_fs)]
    this_ecg_ref = ecg_peaks[(ecg_peaks >= (epoch)* window_fs) * 
                                     (ecg_peaks < (epoch+1) * window_fs)]
    feats = feats.append({'epoch': epoch, 'F1_ppg': bsqi(this_peaks_ref, this_peaks_comp, fs=fs, agw=0.1),
                          'F1_ecg': bsqi(this_ecg_ref, this_peaks_ref, fs=fs, agw=0.1)}, ignore_index=True)

100%|██████████| 1439/1439 [00:02<00:00, 665.34it/s]


In [15]:
from matplotlib import gridspec

ppg_quality = feats.F1_ppg.values
plt.close("all")
fig = plt.figure(figsize=(5, 5)) 
gs = gridspec.GridSpec(2, 1, height_ratios=[1,1.5]) 
ax1 = plt.subplot(gs[0])
t = np.arange(0, len(ppg_quality)*30*256, 30*256)
ax1.plot(t, feats['F1_ppg']*100)
# plt.plot(t, feats['F1_ecg'], '--')
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.set_xticks([])
ax1.set_ylabel('SQI')
ax2 = plt.subplot(gs[1], sharex=ax1)
ax2.plot(ppg)
ax2.plot(ppg_peaks_ref, ppg[ppg_peaks_ref], 'x')
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.set_xlabel('Time (Seconds)')
ax2.set_ylabel('PPG Amplitude')

# plt.plot(ppg_peaks_comp, ppg[ppg_peaks_comp], 'o')
# plt.plot(ecg_peaks, ppg[ecg_peaks], '*')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [75]:
t = np.arange(0, len(ppg)/256, 1/256)
t_q_ppg = np.arange(0, len(ppg_quality)*30*256, 30*256)
t_q_ecg = np.arange(0, len(ecg_quality)*30*256, 30*256)
plt.close("all")
fig, ax = plt.subplots(2, 1, sharex=True)

ax[0].plot(ecg)
# ax[1].plot(ppg)
ax[1].plot(ppg_peaks, ppg[ppg_peaks], 'x')
ax[1].plot(t_q_ppg, ppg_quality)
ax[1].plot(t_q_ecg, ecg_quality, "--")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …